In [ ]:
from pprint import pprint
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv()

llm_name = "gpt-4-1106-preview"
os.environ["OPENAI_API_KEY"]  # you should see the api key if the .env file is loaded correctly

In [2]:
persist_directory = "/Users/dardonacci/Documents/dardonacci/2-Code/metabrain/vector_stores/hummingbot/chroma/"

embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_scripts",
    persist_directory=persist_directory,
    embedding_function=embedding
)

# Prompts

In [3]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. You need to create the full code for the script that the user is asking for. If you think that more information is needed, ask the user for it.
{context}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

In [4]:
llm = ChatOpenAI(model_name=llm_name, temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [5]:
query = """I would like to create a Directional Strategy using Hummingbot using the MACDBB controller. Can you the script for me and suggest values for the indicators?
"""

In [6]:
result = qa_chain({"query": query})

In [7]:
pprint(result["result"])

('Based on your request, you want to create a Directional Strategy using '
 'Hummingbot with the MACDBB controller. Below is a script that sets up a '
 'basic directional strategy using the MACDBB controller. Please note that the '
 'values for the MACD and Bollinger Bands indicators are just examples, and '
 'you should adjust them according to your trading preferences and backtesting '
 'results.\n'
 '\n'
 '```python\n'
 'from decimal import Decimal\n'
 'from typing import Dict, List\n'
 'from collections import deque\n'
 'import datetime\n'
 '\n'
 'from hummingbot.connector.connector_base import ConnectorBase\n'
 'from hummingbot.core.data_type.common import OrderType\n'
 'from hummingbot.data_feed.candles_feed.candles_factory import '
 'CandlesFactory, CandlesConfig\n'
 'from hummingbot.smart_components.controllers.macd_bb_v1 import MACDBBV1, '
 'MACDBBV1Config\n'
 'from hummingbot.strategy.script_strategy_base import ScriptStrategyBase\n'
 'from hummingbot.core.event.events import

In [8]:
len(result["source_documents"])

4